In [3]:
import pandas as pd
import psycopg2

In [56]:
# First, install the psycopg2 package
!pip install psycopg2-binary  # Use this in Jupyter notebook
# or
# !conda install -c conda-forge psycopg2  # Alternative if using conda

# Then import the modules
import pandas as pd
import psycopg2

In [6]:
import pandas as pd
import psycopg2

In [103]:
import os
import json
import nbformat

# Path to the notebook file - updated to use "Downloads" directory
notebook_path = os.path.join("Downloads", "day3_sql_combined_with_creds.ipynb")

# Option 1: Display information about the notebook
with open(notebook_path, 'r', encoding='utf-8') as f:
    notebook_content = json.load(f)
    
print(f"Notebook name: day3_sql_combined_with_creds.ipynb")
print(f"Number of cells: {len(notebook_content['cells'])}")

# Option 2: Extract code cells from the notebook
print("\nCode cells in the notebook:")
code_cells = []
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)
    for i, cell in enumerate(nb.cells):
        if cell.cell_type == 'code':
            code_cells.append(cell.source)
            print(f"\n--- Code Cell {i+1} ---")
            print(cell.source[:200] + "..." if len(cell.source) > 200 else cell.source)

print(f"\nTotal code cells extracted: {len(code_cells)}")

# Note: To execute a specific code cell, you can run:
# exec(code_cells[cell_number])

Notebook name: day3_sql_combined_with_creds.ipynb
Number of cells: 11

Code cells in the notebook:

--- Code Cell 3 ---
import pandas as pd
import psycopg2

--- Code Cell 5 ---
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-gli...

--- Code Cell 7 ---
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

--- Code Cell 9 ---
# Example: Count schools by borough
query = """
SELECT borough, COUNT(*) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough;
"""
df_result = pd.read_sql(query, conn)
df_result

Total code cells extracted: 4


In [104]:
import psycopg2
import pandas as pd

# DB connection setup
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="a9Am7Yy5r9_T7h4OF2GN",  # your correct password
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)

# Create a cursor (optional, for direct SQL execution)
cur = conn.cursor()

# Test the connection
try:
    cur.execute("SELECT 1;")
    print("✅ Connection successful!")
except Exception as e:
    print("❌ Connection failed:", e)


✅ Connection successful!


In [105]:
query = """
SELECT * FROM nyc_schools.high_school_directory LIMIT 5;
""" 
df = pd.read_sql(query, conn)
df.head()

/var/folders/gv/c1xdc57121nf6l68nrddnzwm0000gn/T/ipykernel_25229/3217040630.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [109]:
#How many schools are there in each borough?
query = """
SELECT borough, COUNT(*) AS school_count
FROM nyc_schools.high_school_directory
GROUP BY borough;
"""
df_result = pd.read_sql(query, conn)
df_result

/var/folders/gv/c1xdc57121nf6l68nrddnzwm0000gn/T/ipykernel_25229/524344663.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,school_count
0,Brooklyn,121
1,Queens,80
2,Staten Island,10
3,Manhattan,106
4,Bronx,118


In [114]:
#What is the average % of English Language Learners (ELL) per borough?
query = """
SELECT 
    H.borough,
  ROUND(AVG(S.ell_percent)::numeric, 2) AS avg_ell_percent
FROM 
    nyc_schools.high_school_directory AS H
JOIN 
     nyc_schools.school_demographics AS S
    ON H.dbn = S.dbn
GROUP BY 
    H.borough
ORDER BY 
    avg_ell_percent DESC;
"""
df_result = pd.read_sql(query, conn)
df_result

/var/folders/gv/c1xdc57121nf6l68nrddnzwm0000gn/T/ipykernel_25229/1560769498.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Manhattan,7.57


In [112]:
#Using the data from the school demographics and high school directory, write a query to find the top 3 schools in each borough with the highest percentage of special education students (sped_percent)

query = """
WITH ranked_schools AS (
    SELECT 
        H.borough,
        H.school_name,
        S.sped_percent,
        ROW_NUMBER() OVER (PARTITION BY H.borough ORDER BY S.sped_percent DESC) AS rn
    FROM 
        nyc_schools.high_school_directory AS H
    JOIN 
        nyc_schools.school_demographics AS S
        ON H.dbn = S.dbn
)
SELECT
    borough,
    school_name,
    sped_percent
FROM
    ranked_schools
WHERE
    rn <= 3
ORDER BY
    borough,
    sped_percent DESC;
"""

df_result = pd.read_sql(query, conn)
df_result


/var/folders/gv/c1xdc57121nf6l68nrddnzwm0000gn/T/ipykernel_25229/1771357617.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(query, conn)


,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7


In [ ]:
#Notes to consider (Insights)

#School Distribution & ELL Representation: Brooklyn and the Bronx have the highest number of schools (121 and 118 respectively), while Staten Island has the fewest (10). Manhattan has an average of 7.57% English Language Learners (ELL), highlighting relatively low ELL representation compared to other boroughs.

#Special Education Hotspots: The top schools in Manhattan for special education (sped_percent) reach nearly 29% (28.8), indicating that certain schools serve a significantly higher proportion of students with special needs. This underscores the importance of targeted resources and support in these schools.